### 这里是这版要用的思路，轻量版评论爬取
#### 每个景点只考虑默认的3020条评论
几个要点要考虑
1. 采用多线程池加速爬取

2. cticket两个交替随机使用 

3. 加入 fake_useragent 减少反爬

3. 每一列的变量包括 comment_id, comment_user_id, poi_id, poi_name, comment_time 和 comment_type 

4. 可以先导出评论再加入景点相关的变量 


#### 1.1 加载需要的库

In [ ]:
# -*- coding: utf-8 -*-
import requests
import time
import os
import math
import csv
import ssl
import pandas as pd
from requests.adapters import HTTPAdapter
import json
import random
import re
from fake_useragent import UserAgent 
from concurrent.futures import ThreadPoolExecutor, as_completed

#### 1.2 设置参数

In [2]:
PAGE_SIZE = 20      # 每页条数
MAX_PAGE = 300      # 最多翻多少页（防止过多）
POOL_NUMBER = 6    # 线程池数量
TIME_OUT = 10       # 超时时间

In [3]:
city_name = "北京"

# 创建城市文件夹（如果不存在）
city_folder = os.path.join("comments_result_time", city_name)
os.makedirs(city_folder, exist_ok=True)

In [4]:
# --------- 通用配置 ---------
HEADERS = {
    'authority': 'm.ctrip.com',
    'accept': '*/*',
    'content-type': 'application/json',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'cookieorigin': 'https://you.ctrip.com',
    'origin': 'https://you.ctrip.com',
    'pragma': 'no-cache',
    'referer': 'https://you.ctrip.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
}

ctickets = [
    "E89D6D1A5B866B002944528EAD21DD72A941215F8FEFEA4DC289217D0F196468",
    "A7F9A3EFD9AA82FFB0032491AE748FEDE7F3C0CDE1D26A99545E319742E6D2BD"
]

class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_version'] = ssl.PROTOCOL_TLSv1_2
        return super().init_poolmanager(*args, **kwargs)


In [5]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import glob

In [2]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [3]:
import os
import re
import time
import random
import requests
import pandas as pd
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor, as_completed

# ===== 可修改全局参数 =====
PAGE_SIZE = 20
TIME_OUT = 10
POOL_NUMBER = 10


class SpiderCtripComments:
    def __init__(self, poi_id: int, name: str, folder: str,
                 poi_type: str = "", poi_subtype: str = "",
                 ctickets: list = None, sleep_time: float = 0.3):
        self.poi_id = poi_id
        self.name = name
        self.poi_type = poi_type
        self.poi_subtype = poi_subtype
        safe_name = re.sub(r'[\\/:*?"<>|]', "_", name)
        self.file_path = os.path.join(folder, f"{safe_name}.csv")
        os.makedirs(folder, exist_ok=True)
        self.ua = UserAgent()
        self.ctickets = ctickets or []
        self.sleep_time = sleep_time
        self.seen_comment_ids = set()
        self.sort_folder = os.path.join(folder, "sort_csvs")
        os.makedirs(self.sort_folder, exist_ok=True)

        # ✅ 增加：requests Session + 重试机制
        self.session = requests.Session()
        retries = Retry(
            total=5,                # 最多重试5次
            backoff_factor=1,       # 每次失败等待时间指数增长（1s, 2s, 4s...）
            status_forcelist=[500, 502, 503, 504],  # 对这些错误码自动重试
            allowed_methods=["HEAD", "GET", "OPTIONS", "POST"]  # POST也可以重试
        )
        self.session.mount("https://", HTTPAdapter(max_retries=retries))

        # 加载旧的评论ID
        if os.path.exists(self.file_path):
            try:
                old_df = pd.read_csv(self.file_path, usecols=["commentId"])
                self.seen_comment_ids = set(old_df["commentId"].astype(str).tolist())
                print(f"📂 已加载 {len(self.seen_comment_ids)} 条历史评论用于去重")
            except Exception as e:
                print(f"⚠️ 载入已有 CSV 出错: {e}")

    def get_next_cookie(self):
        if not self.ctickets:
            return {}
        return {"cticket": random.choice(self.ctickets)}

    def generate_params(self):
        guid = str(int(time.time() * 1000000))
        rand = random.randint(100000, 999999)
        return {"_fxpcqlniredt": guid, "x-traceID": f"{guid}-{int(time.time() * 1000000)}-{rand}"}

    # ========== 抓取单页评论 ==========
    def fetch_comments(self, sort_type: int, page_index: int, comment_tag_id: int = 0) -> list:
        url = "https://m.ctrip.com/restapi/soa2/13444/json/getCommentCollapseList"
        params = self.generate_params()
        data = {
            "arg": {
                "channelType": 2,
                "collapseType": 0,
                "commentTagId": int(comment_tag_id),
                "pageIndex": page_index,
                "pageSize": PAGE_SIZE,
                "poiId": int(self.poi_id),
                "resourceType": 11,
                "sortType": int(sort_type),
                "sourceType": 1,
                "starType": 0,
            },
            "head": {
                "auth": "",
                "cid": params["_fxpcqlniredt"],
                "ctok": "",
                "cver": "1.0",
                "lang": "01",
                "sid": "8888",
                "syscode": "09",
                "xsid": "",
            }
        }

        for attempt in range(3):
            try:
                time.sleep(random.uniform(0.5, 1.0))  # 请求前等待，降低封禁概率
                res = self.session.post(
                    url=url,
                    params=params,
                    headers={
                        "User-Agent": self.ua.random,
                        "Content-Type": "application/json;charset=UTF-8"
                    },
                    cookies=self.get_next_cookie(),
                    json=data,
                    timeout=TIME_OUT,
                    verify=True
                )
                res.raise_for_status()
                time.sleep(self.sleep_time + random.uniform(0.1, 0.4))
                j = res.json()
                return (j.get("result") or {}).get("items", []) or []
            except Exception as e:
                print(f"[请求失败] sort={sort_type} tag={comment_tag_id} 页 {page_index}：{e}")
                time.sleep(random.uniform(1, 4))
        return []

    # ========== 抓取单个排序（含多个 tag） ==========
    def crawl_one_sort(self, sort_type: int, max_page: int = 200,
                   threaded: bool = False, batch_size: int = 100,
                   comment_tag_ids: list = [0, -11, -12, -22]):
        safe_name = re.sub(r'[\\/:*?"<>|]', "_", self.name)
        for comment_tag_id in comment_tag_ids:
            tag_suffix = f"_tag{comment_tag_id}" if comment_tag_id != 0 else ""
            out_csv = os.path.join(self.sort_folder, f"{safe_name}_sort{sort_type}{tag_suffix}.csv")

            local_seen = set()
            if os.path.exists(out_csv):
                try:
                    old_ids = pd.read_csv(out_csv, usecols=["commentId"])
                    local_seen = set(old_ids["commentId"].astype(str).tolist())
                    print(f"📂 已加载 sort={sort_type} tag={comment_tag_id} 历史 {len(local_seen)} 条用于去重")
                except Exception as e:
                    print(f"⚠️ 载入 {out_csv} 出错: {e}")

            print(f"\n[抓取 sort={sort_type}, tag={comment_tag_id}] {self.name} (POI: {self.poi_id}) ...")

            def fetch_one_page(page_index):
                items = self.fetch_comments(sort_type, page_index, comment_tag_id)
                return page_index, items

            comments_list = []
            new_count = 0
            empty_pages = 0
            max_consec_empty = 3

            with ThreadPoolExecutor(max_workers=threaded and 10 or 1) as executor:
                futures = {executor.submit(fetch_one_page, i): i for i in range(1, max_page + 1)}
                for future in as_completed(futures):
                    page_index, items = future.result()
                    if not items:
                        empty_pages += 1
                        print(f"⚠️ sort={sort_type} tag={comment_tag_id} 第 {page_index} 页为空")
                        if empty_pages >= max_consec_empty:
                            print("🚫 连续空页，提前停止")
                            break
                        continue

                    empty_pages = 0
                    for it in items:
                        cid = str(it.get("commentId", ""))
                        if not cid or cid in local_seen:
                            continue
                        local_seen.add(cid)
                        comments_list.append({
                            "poi_id": self.poi_id,
                            "poi_name": self.name,
                            "poi_type": self.poi_type,
                            "poi_subtype": self.poi_subtype,
                            "commentId": cid,
                            "content": it.get("content", "").replace("|", " ").replace("\n", " "),
                            "score": it.get("score", ""),
                            "touristTypeDisplay": it.get("touristTypeDisplay", ""),
                            "timeDuration": it.get("timeDuration", ""),
                            "publishTypeTag": it.get("publishTypeTag", ""),
                            "sortType": sort_type,
                            "commentTagId": comment_tag_id
                        })
                        new_count += 1

                    if comments_list:
                        pd.DataFrame(comments_list).to_csv(
                            out_csv, mode="a", index=False, encoding="utf-8-sig",
                            header=not os.path.exists(out_csv)
                        )
                        comments_list.clear()

            print(f"🎉 sort={sort_type} tag={comment_tag_id} 完成：新增 {new_count} 条 → {out_csv}")

    # ========== 合并所有排序 ==========
    def merge_sort_csvs(self):
        safe_name = re.sub(r'[\\/:*?"<>|]', "_", self.name)
        csvs = []
        for st in [1, 2, 3, 4, 5]:
            for tag in [0, -11, -12, -22]:
                path = os.path.join(self.sort_folder, f"{safe_name}_sort{st}_tag{tag}.csv")
                if os.path.exists(path):
                    csvs.append(pd.read_csv(path))
        if not csvs:
            print("⚠️ 没有找到任何 CSV 文件用于合并")
            return
        all_df = pd.concat(csvs, ignore_index=True)
        unique_df = all_df.drop_duplicates(subset="commentId", keep="first")
        out_all = self.file_path.replace(f"{safe_name}.csv", f"{safe_name}_all_unique.csv")
        unique_df.to_csv(out_all, index=False, encoding="utf-8-sig")
        print(f"📦 合并完成：输入 {sum(len(df) for df in csvs)} 条 → 去重后 {len(unique_df)} 条 → {out_all}")

    # ========== 一键导出所有 sort + tag ==========
    def export_by_all_sorts(self, max_page: int = 200, threaded: bool = False, batch_size: int = 100):
        for st in [1, 2, 3, 4, 5]:
            self.crawl_one_sort(sort_type=st, max_page=max_page, threaded=threaded,
                                batch_size=batch_size, comment_tag_ids=[0, -11, -12, -22])
        self.merge_sort_csvs()


In [71]:
if __name__ == "__main__":
    spider = SpiderCtripComments(
        poi_id=80261,
        name="上海科技馆",
        folder="./ctrip_comments_time",
        poi_type="知识科普型",
        poi_subtype="科技馆",
        ctickets = [
        "E89D6D1A5B866B002944528EAD21DD72A941215F8FEFEA4DC289217D0F196468",
        "A7F9A3EFD9AA82FFB0032491AE748FEDE7F3C0CDE1D26A99545E319742E6D2BD"
        ]
    )
    for sort_type in range(1, 6):
        spider.crawl_one_sort(
            sort_type=sort_type,
            threaded=True,
            comment_tag_ids=[0, -11, -12, -22]  # 可根据需要修改
        )

📂 已加载 sort=1 tag=0 历史 3020 条用于去重

[抓取 sort=1, tag=0] 上海科技馆 (POI: 80261) ...
⚠️ sort=1 tag=0 第 155 页为空
⚠️ sort=1 tag=0 第 153 页为空
⚠️ sort=1 tag=0 第 152 页为空
🚫 连续空页，提前停止
🎉 sort=1 tag=0 完成：新增 0 条 → ./ctrip_comments_time\sort_csvs\上海科技馆_sort1.csv

[抓取 sort=1, tag=-11] 上海科技馆 (POI: 80261) ...
⚠️ sort=1 tag=-11 第 154 页为空
⚠️ sort=1 tag=-11 第 152 页为空
⚠️ sort=1 tag=-11 第 153 页为空
⚠️ sort=1 tag=-11 第 155 页为空
⚠️ sort=1 tag=-11 第 156 页为空
🚫 连续空页，提前停止
🎉 sort=1 tag=-11 完成：新增 3020 条 → ./ctrip_comments_time\sort_csvs\上海科技馆_sort1_tag-11.csv

[抓取 sort=1, tag=-12] 上海科技馆 (POI: 80261) ...
⚠️ sort=1 tag=-12 第 24 页为空
⚠️ sort=1 tag=-12 第 22 页为空
⚠️ sort=1 tag=-12 第 23 页为空
🚫 连续空页，提前停止
🎉 sort=1 tag=-12 完成：新增 340 条 → ./ctrip_comments_time\sort_csvs\上海科技馆_sort1_tag-12.csv

[抓取 sort=1, tag=-22] 上海科技馆 (POI: 80261) ...
⚠️ sort=1 tag=-22 第 152 页为空
⚠️ sort=1 tag=-22 第 153 页为空
⚠️ sort=1 tag=-22 第 154 页为空
⚠️ sort=1 tag=-22 第 155 页为空
⚠️ sort=1 tag=-22 第 156 页为空
🚫 连续空页，提前停止
🎉 sort=1 tag=-22 完成：新增 3020 条 → ./ctrip_comments_time\

In [72]:
poi_name = "上海科技馆"
safe_name = "".join(c if c not in r'\/:*?"<>|' else "_" for c in poi_name)
main_folder = "./ctrip_comments_time"
sort_folder = os.path.join(main_folder, "sort_csvs")
output_path = os.path.join(main_folder, f"{safe_name}_all_unique.csv")

# 匹配所有相关 CSV 文件（可根据你实际命名调整通配符）
pattern = os.path.join(sort_folder, f"{safe_name}_sort*.csv")
csv_files = glob.glob(pattern)

# 合并所有文件
if csv_files:
    print(f"🗂️ 共找到 {len(csv_files)} 个文件：")
    for f in csv_files:
        print("   -", os.path.basename(f))

    dfs = []
    for f in csv_files:
        try:
            df = pd.read_csv(f, on_bad_lines="skip", engine="python", encoding="utf-8")
            dfs.append(df)
        except Exception as e:
            print(f"⚠️ 读取 {os.path.basename(f)} 出错: {e}")

    if dfs:
        merged = pd.concat(dfs, ignore_index=True)
        if "commentId" in merged.columns:
            merged.drop_duplicates(subset="commentId", inplace=True)
        merged.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"\n✅ 合并完成，共 {len(merged)} 条评论。")
        print(f"📁 已保存到：{output_path}")
    else:
        print("❌ 未读取到有效数据。")
else:
    print("⚠️ 未找到匹配的 CSV 文件，请检查 sort_folder 路径和文件命名。")

🗂️ 共找到 20 个文件：
   - 上海科技馆_sort1.csv
   - 上海科技馆_sort1_tag-11.csv
   - 上海科技馆_sort1_tag-12.csv
   - 上海科技馆_sort1_tag-22.csv
   - 上海科技馆_sort2.csv
   - 上海科技馆_sort2_tag-11.csv
   - 上海科技馆_sort2_tag-12.csv
   - 上海科技馆_sort2_tag-22.csv
   - 上海科技馆_sort3.csv
   - 上海科技馆_sort3_tag-11.csv
   - 上海科技馆_sort3_tag-12.csv
   - 上海科技馆_sort3_tag-22.csv
   - 上海科技馆_sort4.csv
   - 上海科技馆_sort4_tag-11.csv
   - 上海科技馆_sort4_tag-12.csv
   - 上海科技馆_sort4_tag-22.csv
   - 上海科技馆_sort5.csv
   - 上海科技馆_sort5_tag-11.csv
   - 上海科技馆_sort5_tag-12.csv
   - 上海科技馆_sort5_tag-22.csv

✅ 合并完成，共 11817 条评论。
📁 已保存到：./ctrip_comments_time\上海科技馆_all_unique.csv
